# MNIST 手寫數字辨識 - Convolution Neural Network

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

載入 MNIST 資料集

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

im_dim = mnist.train.images.shape[1] # 影像維度
lb_dim = mnist.train.labels.shape[1] # 類別數量

x  = tf.placeholder(tf.float32, [None, im_dim]) # 輸入資料
y_ = tf.placeholder(tf.float32, [None, lb_dim]) # 實際結果

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


將影像資料轉為四維資料 [numbers, width, height, channels]

In [3]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) # -1 means automatically compute how many sets

## 建立 Convolution Neural Network

定義權重、偏差值初始化

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1) # stddev = 標準差
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

定義卷積層 與 池化層

In [5]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'SAME') # strides = [1, x_movement, y_movement, 1]

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

### First Convolution Layer

In [6]:
W_conv1 = weight_variable([5, 5, 1, 32]) # [patch_width, patch_height, channels, feature map]
b_conv1 = bias_variable([32]) # numbers of feature map
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # output size 28x28x32
h_pool1 = max_pool_2x2(h_conv1) # output size 14x14x32

### Second Convolution Layer

In [7]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 14x14x64
h_pool2 = max_pool_2x2(h_conv2) # output size 7x7x64

將第二卷積層的輸出轉為一維資料，用於連接之後的全連接層

In [8]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64]) # [n_samples, 7, 7, 64] -> [n_samples, 7 * 7 * 64]

### Fully Connective Layer

In [9]:
W_fc1 = weight_variable([7 * 7 * 64, 1024]) # input size 7*7*64, output size 1024
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) # output size 1024

### Dropout

In [10]:
keep_prob  = tf.placeholder(tf.float32) # 決定神經元不變動的機率
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Output Layer

In [11]:
W_fc2 = weight_variable([1024, 10]) # input size 1024, output size 10
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2) # 輸出結果

## 訓練與評估模型

定義 loss function 為 cross_entropy

In [12]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices = [1]))

定義最佳化方法為 ADAM最佳化演算法

In [13]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

 定義評估方法

In [14]:
correct_predict = tf.equal(tf.argmax(y_, 1), tf.argmax(y_conv, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))

建立模型儲存器

In [15]:
saver = tf.train.Saver()

初始化參數

In [16]:
init = tf.global_variables_initializer()

啟動 Graph

In [17]:
sess = tf.Session()
sess.run(init)

### 訓練模型 & 顯示訓練過程

In [18]:
itera     = 2000 # 訓練次數
num_batch = 50   # 訓練的批次量
for step in range(itera):
    [batch_xs, batch_ys] = mnist.train.next_batch(num_batch)
    sess.run(train_step, feed_dict = {x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    
    if step % 50 == 0:
        print('第 {0} 次 測試準確率: {1:.2f}'.format(step, sess.run(accuracy,feed_dict = {x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1})))

第 0 次 測試準確率: 0.10
第 50 次 測試準確率: 0.71
第 100 次 測試準確率: 0.83
第 150 次 測試準確率: 0.87
第 200 次 測試準確率: 0.89
第 250 次 測試準確率: 0.91
第 300 次 測試準確率: 0.91
第 350 次 測試準確率: 0.92
第 400 次 測試準確率: 0.92
第 450 次 測試準確率: 0.94
第 500 次 測試準確率: 0.94
第 550 次 測試準確率: 0.94
第 600 次 測試準確率: 0.94
第 650 次 測試準確率: 0.95
第 700 次 測試準確率: 0.95
第 750 次 測試準確率: 0.95
第 800 次 測試準確率: 0.95
第 850 次 測試準確率: 0.96
第 900 次 測試準確率: 0.96
第 950 次 測試準確率: 0.96
第 1000 次 測試準確率: 0.96
第 1050 次 測試準確率: 0.96
第 1100 次 測試準確率: 0.96
第 1150 次 測試準確率: 0.96
第 1200 次 測試準確率: 0.96
第 1250 次 測試準確率: 0.96
第 1300 次 測試準確率: 0.96
第 1350 次 測試準確率: 0.97
第 1400 次 測試準確率: 0.97
第 1450 次 測試準確率: 0.97
第 1500 次 測試準確率: 0.97
第 1550 次 測試準確率: 0.97
第 1600 次 測試準確率: 0.97
第 1650 次 測試準確率: 0.97
第 1700 次 測試準確率: 0.97
第 1750 次 測試準確率: 0.97
第 1800 次 測試準確率: 0.97
第 1850 次 測試準確率: 0.97
第 1900 次 測試準確率: 0.97
第 1950 次 測試準確率: 0.97


## 儲存模型

In [20]:
save_model = saver.save(sess, ('/models/mnist_cnn_%d.ckpt' % (itera)))